In [1]:
# breakhistf2v29.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y
 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    
    layers.Dense(128), 
    layers.Dropout(rate=0.5),      
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v29' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=1, epochs=2000, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0

Epoch 45/2000
297/297 [==============================] - 246s 829ms/step - loss: 0.8772 - accuracy: 0.6789 - val_loss: 0.8980 - val_accuracy: 0.6719
Epoch 46/2000
297/297 [==============================] - 246s 829ms/step - loss: 0.8706 - accuracy: 0.6754 - val_loss: 0.8905 - val_accuracy: 0.6685
Epoch 47/2000
297/297 [==============================] - 246s 830ms/step - loss: 0.8639 - accuracy: 0.6818 - val_loss: 0.8856 - val_accuracy: 0.6713
Epoch 48/2000
297/297 [==============================] - 247s 833ms/step - loss: 0.8522 - accuracy: 0.6818 - val_loss: 0.8938 - val_accuracy: 0.6650
Epoch 49/2000
297/297 [==============================] - 249s 839ms/step - loss: 0.8574 - accuracy: 0.6890 - val_loss: 0.9461 - val_accuracy: 0.6574
Epoch 50/2000
297/297 [==============================] - 248s 834ms/step - loss: 0.8427 - accuracy: 0.6930 - val_loss: 0.8733 - val_accuracy: 0.6656
Epoch 51/2000
297/297 [==============================] - 248s 835ms/step - loss: 0.8386 - accuracy: 0.6891

Epoch 100/2000
297/297 [==============================] - 268s 901ms/step - loss: 0.6558 - accuracy: 0.7475 - val_loss: 0.7579 - val_accuracy: 0.7133
Epoch 101/2000
297/297 [==============================] - 264s 889ms/step - loss: 0.6643 - accuracy: 0.7470 - val_loss: 0.7965 - val_accuracy: 0.7048
Epoch 102/2000
297/297 [==============================] - 266s 895ms/step - loss: 0.6523 - accuracy: 0.7540 - val_loss: 0.7527 - val_accuracy: 0.7124
Epoch 103/2000
297/297 [==============================] - 266s 896ms/step - loss: 0.6589 - accuracy: 0.7509 - val_loss: 0.7706 - val_accuracy: 0.7111
Epoch 104/2000
297/297 [==============================] - 267s 899ms/step - loss: 0.6476 - accuracy: 0.7534 - val_loss: 0.7514 - val_accuracy: 0.7231
Epoch 105/2000
297/297 [==============================] - 267s 899ms/step - loss: 0.6434 - accuracy: 0.7604 - val_loss: 0.7526 - val_accuracy: 0.7168
Epoch 106/2000
297/297 [==============================] - 263s 886ms/step - loss: 0.6478 - accuracy:

Epoch 209/2000
297/297 [==============================] - 247s 831ms/step - loss: 0.4795 - accuracy: 0.8149 - val_loss: 0.7285 - val_accuracy: 0.7298
Epoch 210/2000
297/297 [==============================] - 251s 845ms/step - loss: 0.4718 - accuracy: 0.8163 - val_loss: 0.7603 - val_accuracy: 0.7282
Epoch 211/2000
297/297 [==============================] - 249s 838ms/step - loss: 0.4788 - accuracy: 0.8152 - val_loss: 0.7375 - val_accuracy: 0.7345
Epoch 212/2000
297/297 [==============================] - 250s 840ms/step - loss: 0.4822 - accuracy: 0.8156 - val_loss: 0.7358 - val_accuracy: 0.7389
Epoch 213/2000
297/297 [==============================] - 249s 839ms/step - loss: 0.4802 - accuracy: 0.8107 - val_loss: 0.7522 - val_accuracy: 0.7415
Epoch 214/2000
297/297 [==============================] - 248s 834ms/step - loss: 0.4746 - accuracy: 0.8161 - val_loss: 0.6927 - val_accuracy: 0.7449
Epoch 215/2000
297/297 [==============================] - 249s 837ms/step - loss: 0.4573 - accuracy:

Epoch 318/2000
297/297 [==============================] - 248s 834ms/step - loss: 0.3844 - accuracy: 0.8505 - val_loss: 0.7207 - val_accuracy: 0.7557
Epoch 319/2000
297/297 [==============================] - 247s 830ms/step - loss: 0.3756 - accuracy: 0.8549 - val_loss: 0.7317 - val_accuracy: 0.7544
Epoch 320/2000
297/297 [==============================] - 247s 833ms/step - loss: 0.3672 - accuracy: 0.8575 - val_loss: 0.7443 - val_accuracy: 0.7535
Epoch 321/2000
297/297 [==============================] - 255s 860ms/step - loss: 0.3871 - accuracy: 0.8477 - val_loss: 0.6990 - val_accuracy: 0.7551
Epoch 322/2000
297/297 [==============================] - 265s 891ms/step - loss: 0.3719 - accuracy: 0.8610 - val_loss: 0.7207 - val_accuracy: 0.7579
Epoch 323/2000
297/297 [==============================] - 262s 883ms/step - loss: 0.3575 - accuracy: 0.8624 - val_loss: 0.7013 - val_accuracy: 0.7601
Epoch 324/2000
297/297 [==============================] - 250s 843ms/step - loss: 0.3605 - accuracy:

...

297/297 [==============================] - 273s 920ms/step - loss: 0.1519 - accuracy: 0.9411 - val_loss: 0.9816 - val_accuracy: 0.7816
Epoch 1621/2000
297/297 [==============================] - 268s 904ms/step - loss: 0.1509 - accuracy: 0.9405 - val_loss: 0.9487 - val_accuracy: 0.7800
Epoch 1622/2000
297/297 [==============================] - 273s 920ms/step - loss: 0.1328 - accuracy: 0.9481 - val_loss: 0.9272 - val_accuracy: 0.7898
Epoch 1623/2000
297/297 [==============================] - 268s 902ms/step - loss: 0.1520 - accuracy: 0.9438 - val_loss: 1.0203 - val_accuracy: 0.7693
Epoch 1624/2000
297/297 [==============================] - 267s 899ms/step - loss: 0.1450 - accuracy: 0.9452 - val_loss: 1.0206 - val_accuracy: 0.7876
Epoch 1625/2000
297/297 [==============================] - 269s 905ms/step - loss: 0.1345 - accuracy: 0.9475 - val_loss: 0.9457 - val_accuracy: 0.7819
Epoch 1626/2000
297/297 [==============================] - 256s 860ms/step - loss: 0.1374 - accuracy: 0.9450 -

297/297 [==============================] - 265s 891ms/step - loss: 0.1371 - accuracy: 0.9453 - val_loss: 0.9743 - val_accuracy: 0.7803
Epoch 1729/2000
297/297 [==============================] - 268s 903ms/step - loss: 0.1383 - accuracy: 0.9460 - val_loss: 0.9558 - val_accuracy: 0.7905
Epoch 1730/2000
297/297 [==============================] - 266s 895ms/step - loss: 0.1385 - accuracy: 0.9483 - val_loss: 0.9582 - val_accuracy: 0.7933
Epoch 1731/2000
297/297 [==============================] - 266s 896ms/step - loss: 0.1441 - accuracy: 0.9429 - val_loss: 0.9993 - val_accuracy: 0.7740
Epoch 1732/2000
297/297 [==============================] - 266s 896ms/step - loss: 0.1449 - accuracy: 0.9445 - val_loss: 0.9564 - val_accuracy: 0.7835
Epoch 1733/2000
297/297 [==============================] - 267s 899ms/step - loss: 0.1264 - accuracy: 0.9502 - val_loss: 0.9104 - val_accuracy: 0.7867
Epoch 1734/2000
297/297 [==============================] - 267s 900ms/step - loss: 0.1427 - accuracy: 0.9445 -

297/297 [==============================] - 265s 891ms/step - loss: 0.1278 - accuracy: 0.9523 - val_loss: 1.0184 - val_accuracy: 0.7753
Epoch 1837/2000
297/297 [==============================] - 265s 894ms/step - loss: 0.1387 - accuracy: 0.9463 - val_loss: 1.0589 - val_accuracy: 0.7769
Epoch 1838/2000
297/297 [==============================] - 265s 894ms/step - loss: 0.1356 - accuracy: 0.9459 - val_loss: 1.0331 - val_accuracy: 0.7784
Epoch 1839/2000
297/297 [==============================] - 264s 890ms/step - loss: 0.1248 - accuracy: 0.9538 - val_loss: 1.0037 - val_accuracy: 0.7841
Epoch 1840/2000
297/297 [==============================] - 268s 903ms/step - loss: 0.1286 - accuracy: 0.9494 - val_loss: 1.0185 - val_accuracy: 0.7797
Epoch 1841/2000
297/297 [==============================] - 249s 838ms/step - loss: 0.1332 - accuracy: 0.9487 - val_loss: 1.0748 - val_accuracy: 0.7623
Epoch 1842/2000
297/297 [==============================] - 251s 845ms/step - loss: 0.1406 - accuracy: 0.9454 -

297/297 [==============================] - 267s 900ms/step - loss: 0.1329 - accuracy: 0.9482 - val_loss: 0.9964 - val_accuracy: 0.7807
Epoch 1945/2000
297/297 [==============================] - 269s 905ms/step - loss: 0.1321 - accuracy: 0.9495 - val_loss: 1.0494 - val_accuracy: 0.7886
Epoch 1946/2000
297/297 [==============================] - 269s 905ms/step - loss: 0.1296 - accuracy: 0.9493 - val_loss: 0.9637 - val_accuracy: 0.7914
Epoch 1947/2000
297/297 [==============================] - 268s 903ms/step - loss: 0.1336 - accuracy: 0.9476 - val_loss: 1.0296 - val_accuracy: 0.7816
Epoch 1948/2000
297/297 [==============================] - 268s 904ms/step - loss: 0.1279 - accuracy: 0.9511 - val_loss: 0.9970 - val_accuracy: 0.7939
Epoch 1949/2000
297/297 [==============================] - 270s 908ms/step - loss: 0.1263 - accuracy: 0.9512 - val_loss: 0.9955 - val_accuracy: 0.7845
Epoch 1950/2000
297/297 [==============================] - 268s 902ms/step - loss: 0.1364 - accuracy: 0.9493 -

[1.1445358765125275, 0.7749684]